In [1]:
# DATA EXTRACTION FROM THE ORIGINAL FILES

In [25]:
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

In [26]:
pwd

'/home/purva/Desktop/purva/Single_shot_detector'

In [27]:
import numpy as np

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas
import os
import warnings
import sklearn.utils
from copy import deepcopy
from PIL import Image
from bs4 import BeautifulSoup
import time
import csv
import random
import sys
import pickle
from tqdm import tqdm, trange

from SSD_VGG16 import SSDBoxEncoder

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11815179343562343161
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 715128900066388826
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5996905616586890750
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17021470034951504829
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3634109189779882519
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:3"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12503219660809635503
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type:

NameError: name 'pwd' is not defined

In [28]:
VOC_2007_images_dir      = "./VOC2007/JPEGImages/"
VOC_2007_test_images_dir = './VOC2007_Test/JPEGImages/'
VOC_2012_images_dir      = './VOC2012/JPEGImages/'

# The directories that contain the annotations.
VOC_2007_annotations_dir      = './VOC2007/Annotations/'
VOC_2007_test_annotations_dir = './VOC2007_Test/Annotations/'
VOC_2012_annotations_dir      = './VOC2012/Annotations/'

# The paths to the image sets.
VOC_2007_train_image_set_filename    = './VOC2007/ImageSets/Main/train.txt'
VOC_2012_train_image_set_filename    = './VOC2012/ImageSets/Main/train.txt'
VOC_2007_val_image_set_filename      = './VOC2007/ImageSets/Main/val.txt'
VOC_2012_val_image_set_filename      = './VOC2012/ImageSets/Main/val.txt'
VOC_2007_trainval_image_set_filename = './VOC2007/ImageSets/Main/trainval.txt'
VOC_2012_trainval_image_set_filename = './VOC2012/ImageSets/Main/trainval.txt'
VOC_2007_test_image_set_filename     = './VOC2007_Test/ImageSets/Main/test.txt'


In [29]:
pwd

'/home/purva/Desktop/purva/Single_shot_detector'

In [30]:
classes=['background',
                       'aeroplane', 'bicycle', 'bird', 'boat',
                       'bottle', 'bus', 'car', 'cat',
                       'chair', 'cow', 'diningtable', 'dog',
                       'horse', 'motorbike', 'person', 'pottedplant',
                       'sheep', 'sofa', 'train', 'tvmonitor']

In [31]:
def parse_xml(images_dirs,
              image_set_filenames,
              annotations_dirs=[],
              include_classes = 'all',
              exclude_truncated=False,
              exclude_difficult=False):

    box_output_format=['class_id', 'xmin', 'ymin', 'xmax', 'ymax']
    filenames = []
    image_ids = []
    labels = []
    for images_dir, image_set_filename, annotations_dir in zip(images_dirs, image_set_filenames, annotations_dirs):
        # Read the image set file that so that we know all the IDs of all the images to be included in the dataset.
        with open(image_set_filename) as f:
            image_ids = [line.strip() for line in f] # Note: These are strings, not integers.
            image_ids += image_ids

        # Loop over all images in this dataset.
        for image_id in image_ids:

            filename = '{}'.format(image_id) + '.jpg'
            filenames.append(os.path.join(images_dir, filename))

            if not annotations_dir is None:
                # Parse the XML file for this image.
                with open(os.path.join(annotations_dir, image_id + '.xml')) as f:
                    soup = BeautifulSoup(f, 'xml')

                folder = soup.folder.text # In case we want to return the folder in addition to the image file name. Relevant for determining which dataset an image belongs to.
                #filename = soup.filename.text

                boxes = [] # We'll store all boxes for this image here
                objects = soup.find_all('object') # Get a list of all objects in this image

                # Parse the data for each object
                for obj in objects:
                    class_name = obj.find('name').text
                    class_id = classes.index(class_name)
                    # Check if this class is supposed to be included in the dataset
                    
                    pose = obj.pose.text
                    truncated = int(obj.truncated.text)
                    if exclude_truncated and (truncated == 1): continue
                    difficult = int(obj.difficult.text)
                    if exclude_difficult and (difficult == 1): continue
                    xmin = int(obj.bndbox.xmin.text)
                    ymin = int(obj.bndbox.ymin.text)
                    xmax = int(obj.bndbox.xmax.text)
                    ymax = int(obj.bndbox.ymax.text)
                    item_dict = {'folder': folder,
                                 'image_name': filename,
                                 'image_id': image_id,
                                 'class_name': class_name,
                                 'class_id': class_id,
                                 'pose': pose,
                                 'truncated': truncated,
                                 'difficult': difficult,
                                 'xmin': xmin,
                                 'ymin': ymin,
                                 'xmax': xmax,
                                 'ymax': ymax}
                    box = []
                    for item in box_output_format:
                        box.append(item_dict[item])
                    boxes.append(box)

                labels.append(boxes)

    return filenames, labels, image_ids



In [22]:
%%time
train_data=parse_xml(images_dirs=[VOC_2007_images_dir,
                                     VOC_2012_images_dir],
                       
                     image_set_filenames=[VOC_2007_trainval_image_set_filename,
                                             VOC_2012_trainval_image_set_filename],
                     
                     annotations_dirs=[VOC_2007_annotations_dir,
                                          VOC_2012_annotations_dir],
                        
                     
                     exclude_truncated=False,
                     
                     exclude_difficult=False)
train_data

CPU times: user 1min 9s, sys: 259 ms, total: 1min 9s
Wall time: 1min 9s


In [23]:

val_data = parse_xml(images_dirs=[VOC_2007_test_images_dir],
                     
                     image_set_filenames=[VOC_2007_test_image_set_filename],
                     
                     annotations_dirs=[VOC_2007_test_annotations_dir],
                      
                      exclude_truncated=False,
                     
                      exclude_difficult=True)

In [24]:
len(train_data[2])

NameError: name 'train_data' is not defined

# SAVE DATA

In [ ]:
%%time
filenames_out=open("train_filenames.pickle",'wb')
pickle.dump(train_data[0],filenames_out)
filenames_out.close()

In [ ]:
%%time
filenames_out=open("val_filenames.pickle",'wb')
pickle.dump(val_data[0],filenames_out)
filenames_out.close()

In [ ]:
%%time
labels_out=open("train_labels.pickle",'wb')
pickle.dump(train_data[1],labels_out)
labels_out.close()

In [ ]:
%%time
labels_out=open("val_labels.pickle",'wb')
pickle.dump(val_data[1],labels_out)
labels_out.close()

In [ ]:
%%time
image_ids_out=open("train_image_ids.pickle",'wb')
pickle.dump(train_data[2],image_ids_out)
image_ids_out.close()

In [ ]:
%%time
image_ids_out=open("val_image_ids.pickle",'wb')
pickle.dump(val_data[2],image_ids_out)
image_ids_out.close()

# Transformations

In [ ]:
def translate(image, horizontal=(0,40), vertical=(0,10)):
   
    rows,cols,ch = image.shape

    x = np.random.randint(horizontal[0], horizontal[1]+1)
    y = np.random.randint(vertical[0], vertical[1]+1)
    x_shift = random.choice([-x, x])
    y_shift = random.choice([-y, y])

    M = np.float32([[1,0,x_shift],[0,1,y_shift]])
    return cv2.warpAffine(image, M, (cols, rows)), x_shift, y_shift

In [ ]:
import cv2
img=cv2.imread(train_data[0][0])
trans=translate(img)
trans[1]

In [ ]:
#Translated IMAGE

In [ ]:
plt.imshow(translate(img)[0])

In [ ]:
#ORIGINAL IMAGE

In [ ]:
plt.imshow(img)

# FLIP

In [ ]:
def flip(image, orientation='horizontal'):
    
    if orientation == 'horizontal':
        return cv2.flip(image, 1)
    else:
        return cv2.flip(image, 0)

In [ ]:
img=cv2.imread(val_data[0][123])

In [ ]:
print("Original Image")
plt.imshow(img)

In [ ]:
plt.imshow(flip(img))

# Scale

In [ ]:
def scale(image, min=0.9, max=1.1):
    rows,cols,ch = image.shape
    scale = np.random.uniform(min, max)
    M = cv2.getRotationMatrix2D((cols/2,rows/2), 0, scale)
    return cv2.warpAffine(image, M, (cols, rows)), M, scale


In [ ]:
img=cv2.imread(val_data[0][111])
print("Original Image:")
plt.imshow(img)

In [ ]:

print("Scaled Image:")
plt.imshow(scale(img)[0])

# Brightness

In [ ]:
def brightness(image, min=0.5, max=2.0):
    hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_br = np.random.uniform(min,max)
    mask = hsv[:,:,2] * random_br > 255
    v_channel = np.where(mask, 255, hsv[:,:,2] * random_br)
    hsv[:,:,2] = v_channel
    return cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)


In [ ]:
print("Original Image:")
img=cv2.imread(train_data[0][890])
plt.imshow(img)

In [ ]:
print("Brightened Image:")
img=cv2.imread(train_data[0][890])
plt.imshow(brightness(img))

# Histogram Equlaizer

In [ ]:
def histogram_eq(image):
    image1 = np.copy(image)
    image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2HSV)
    image1[:,:,2] = cv2.equalizeHist(image1[:,:,2])
    image1 = cv2.cvtColor(image1, cv2.COLOR_HSV2RGB)
    return image1

In [ ]:
print("Original Image:")
img=cv2.imread(train_data[0][1890])
plt.imshow(img)

In [ ]:
print("Brightened Image:")
img=cv2.imread(train_data[0][1890])
plt.imshow(histogram_eq(img))